# Data Exploration

In [1]:
import os #For file paths
import keras_cv #For audio spectrograms and data augmentation. This has some prebuilt models
import keras#Main deep learning framework you’re using to build models, layers, and training loops
import keras.backend as K #“Low-level” backend ops used inside/around Keras models
import tensorflow as tf #TensorFlow is one of the possible “backends” Keras can run on,
import tensorflow_io as tfio #For audio tasks,
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers.schedules import CosineDecay

import numpy as np 
import pandas as pd

from glob import glob
from tqdm import tqdm

import librosa #Core audio processing library in Python.
import IPython.display as ipd #For Jupyter/Colab display utilities.
import librosa.display as lid #Plotting helpers for audio from librosa.

import matplotlib.pyplot as plt
import matplotlib as mpl

import random
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
os.environ["KERAS_BACKEND"] = "torch"  # "jax" or "tensorflow" or "torch" 

from collections import defaultdict

2025-12-16 17:48:38.646383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765907318.870562      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765907318.937665      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl8str_util8EndsWithESt17basic_string_viewIcSt11char_traitsIcEES4_']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.11/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZN3tsl8str_util9LowercaseB5cxx11ESt17basic_s

In [2]:
train_meta = pd.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv')
eBird_Taxonomy = pd.read_csv('/kaggle/input/birdclef-2024/eBird_Taxonomy_v2021.csv')

In [3]:
#Configuration class to avoid having configuration features accidentally modified
class CFG:
    seed = 42
    
    # Input image size and batch size
    img_size = [256, 944]
    batch_size = 64
    
    # Audio duration, sample rate, and length
    duration = 10
    sample_rate = 32000 #32 kHz were the downsampled dimensions
    audio_len = duration * sample_rate #douration * sample rate (samples per second)
    
    # STFT parameters
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    
    # Number of epochs, model name
    epochs = 10
    preset = 'efficientnetv2_b2_imagenet'

    patch_size = 32
    batch_size = 8
    epochs = 30
    learning_rate = 1e-4
    
    # Data augmentation parameters
    augment=True

    # Class Labels for BirdCLEF 24, these are actually bird species (common names)
    class_names = sorted(os.listdir('/kaggle/input/birdclef-2024/train_audio/'))
    num_classes = len(class_names)
    class_labels = list(range(num_classes))
    label2name = dict(zip(class_labels, class_names))
    name2label = {specs:idx for idx,specs in label2name.items()}
    spec_dir = "/kaggle/input/bird-call-spectrogram/spec_train"
    train_meta = "/kaggle/input/birdclef-2024/train_metadata.csv"

CFG = CFG()

In [4]:
BASE_PATH = '/kaggle/input/birdclef-2024'
df = pd.read_csv(f'{BASE_PATH}/train_metadata.csv')
df['filepath'] = BASE_PATH + '/train_audio/' + df.filename #Get filepath for the each record
df['target'] = df.primary_label.map(CFG.name2label) #number for each species
df['filename'] = df.filepath.map(lambda x: x.split('/')[-1])
df['xc_id'] = df.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
class_names = sorted(os.listdir(CFG.spec_dir))
num_classes = len(class_names)

name2label = {name: i for i, name in enumerate(class_names)}
label2name = {i: name for name, i in name2label.items()}

print(f"Classes: {num_classes}")


# Display rwos
df

Classes: 182


,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,filepath,target,xc_id
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,XC134896.ogg,/kaggle/input/birdclef-2024/train_audio/asbfly...,0,XC134896
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,XC164848.ogg,/kaggle/input/birdclef-2024/train_audio/asbfly...,0,XC164848
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,XC175797.ogg,/kaggle/input/birdclef-2024/train_audio/asbfly...,0,XC175797
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,XC207738.ogg,/kaggle/input/birdclef-2024/train_audio/asbfly...,0,XC207738
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,XC209218.ogg,/kaggle/input/birdclef-2024/train_audio/asbfly...,0,XC209218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24454,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://xeno-canto.org/845747,XC845747.ogg,/kaggle/input/birdclef-2024/train_audio/zitcis...,181,XC845747
24455,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/845817,XC845817.ogg,/kaggle/input/birdclef-2024/train_audio/zitcis...,181,XC845817
24456,zitcis1,[],[''],51.1207,4.5607,Cisticola juncidis,Zitting Cisticola,Wim Jacobs,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/856176,XC856176.ogg,/kaggle/input/birdclef-2024/train_audio/zitcis...,181,XC856176
24457,zitcis1,[],[''],41.5607,-8.4236,Cisticola juncidis,Zitting Cisticola,Jorge Leitão,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://xeno-canto.org/856723,XC856723.ogg,/kaggle/input/birdclef-2024/train_audio/zitcis...,181,XC856723


In [5]:
#Dictionary of species:
labels = df['primary_label'].unique()
species = df['common_name'].unique()
label2species = dict(zip(labels, species))

In [6]:
#Species Visualization:
print(label2species['pursun4'], len(label2species))

Purple Sunbird 182


# Pre-process audio 

In [7]:
AUDIO_DIR = "/kaggle/input/birdclef-2024/train_audio"
OUT_DIR = "/kaggle/working/spec_train"
SPEC_DIR = '/kaggle/input/bird-call-spectrogram/spec_train'

# Dataset

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

# 1. Gather all file paths and labels
all_npy_paths = []
all_labels = []
SEED = 42

# Walk through the output directory to get files and species labels
for species in os.listdir(SPEC_DIR):
    species_dir = os.path.join(SPEC_DIR, species)
    if os.path.isdir(species_dir):
        
        # Get label index from species name
        if species in CFG.name2label:
            label_idx = CFG.name2label[species]
            for fname in os.listdir(species_dir):
                if fname.endswith('.npy'):
                    all_npy_paths.append(os.path.join(species_dir, fname))
                    all_labels.append(label_idx)

In [9]:
path = "/kaggle/input/bird-call-spectrogram/spec_train/asbfly/XC134896.npy"

spec = np.load(path)

print(spec.shape)

(256, 944)


In [10]:
# 2. Split into Train and Validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    all_npy_paths,
    all_labels,
    test_size=0.2,
    random_state=SEED,
    stratify=all_labels,
)
def load_npy(path, label):
    spec = np.load(path.decode())
    spec = spec.astype(np.float32)
    spec = spec[..., np.newaxis]   # (H, W, 1)
    return spec, label

def tf_load_npy(path, label):
    spec, label = tf.numpy_function(
        load_npy,
        [path, label],
        [tf.float32, tf.int32],
    )
    spec.set_shape([CFG.img_size[0], CFG.img_size[1], 1])
    label.set_shape([])
    return spec, label


In [11]:
# idx = 100
# spec, label = load_npy(val_paths[idx], val_labels[idx])
# print(f"Spectrogram Shape: {spec.shape}")
# print(f"Species class label/No.: {label} | codename: {CFG.label2name[label]} | Common Name: {label2species[CFG.label2name[label]]}")

# Dataset Creation

In [12]:
# 4. Create Tensorflow Datasets
def create_dataset(paths, labels, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(tf_load_npy, num_parallel_calls=tf.data.AUTOTUNE)
    #ds = ds.enumerate()
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(CFG.batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = create_dataset(train_paths, train_labels, shuffle=True)
val_ds = create_dataset(val_paths, val_labels, shuffle=False)

I0000 00:00:1765907340.864750      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765907340.865385      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [13]:
# for idx, (x, y) in train_ds.take(1):
#     print(idx.shape)  # (batch_size,)
#     print(x.shape)    # (batch_size, H, W, 1)

# Patch Creation

In [14]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dim = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dim])
        return patches


In [15]:
# ViT Hyperparameters
# PATCH_SIZE = 16    # Size of the patches to be extracted from the input images
# NUM_PATCHES = (CFG.img_size[0] // PATCH_SIZE) * (CFG.img_size[1] // PATCH_SIZE)
# EMBEDDING_DIM  = [512, 256]
# HIDDEN_UNITS = 4
# NOISE_FACTOR = 0.5

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches,
            output_dim=projection_dim,
        )

    def call(self, patches):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patches)
        encoded += self.position_embedding(positions)
        return encoded


class PatchReconstruction(layers.Layer):
    def __init__(self, patch_size, img_height, img_width, channels=1, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size
        self.img_height = img_height
        self.img_width = img_width
        self.channels = channels
        self.num_patches_h = img_height // patch_size
        self.num_patches_w = img_width // patch_size
    
    def call(self, patches):
        batch_size = tf.shape(patches)[0]
        
        # Reshape patches back to grid
        # patches shape: (batch, num_patches_h, num_pathces_w, patch_size_h, patch_size_w, channels)
        patches = tf.reshape(patches,
                             [batch_size, self.num_patches_h, self.num_patches_w,
                              self.patch_size, self.patch_size, self.channels])
        
        # Rearrange to form image
        # (batch, num_patches_h, patch_size_h, num_pathces_w, patch_size_w, channels)
        patches = tf.transpose(patches, [0, 1, 3, 2, 4, 5])
        
        # Merge patches into image
        images = tf.reshape(patches, [batch_size, self.img_height, self.img_width, self.channels])
        
        return images
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "patch_size": self.patch_size,
            "img_height": self.img_height,
            "img_width": self.img_width,
            "channels": self.channels
        })
        return config

In [16]:
# class PatchDecoder(layers.Layer):
#     """Reconstruct image from patches"""
#     def __init__(self, patch_size, img_height, img_width, channels=1, **kwargs):
#         super().__init__(**kwargs)
#         self.patch_size = patch_size
#         self.img_height = img_height
#         self.img_width = img_width
#         self.channels = channels
#         self.num_patches_h = img_height // patch_size
#         self.num_patches_w = img_width // patch_size
    
#     def call(self, patches):
#         batch_size = tf.shape(patches)[0]
        
#         # Reshape patches back to grid
#         patches = tf.reshape(
#             patches,
#             [batch_size, self.num_patches_h, self.num_patches_w,
#              self.patch_size, self.patch_size, self.channels]
#         )
        
#         # Rearrange to form image
#         patches = tf.transpose(patches, [0, 1, 3, 2, 4, 5])
        
#         # Merge patches into image
#         images = tf.reshape(
#             patches,
#             [batch_size, self.img_height, self.img_width, self.channels]
#         )
        
#         return images
    
#     def get_config(self):
#         config = super().get_config()
#         config.update({
#             "patch_size": self.patch_size,
#             "img_height": self.img_height,
#             "img_width": self.img_width,
#             "channels": self.channels
#         })
#         return config

## Patch Creation - Test

In [17]:
# npy_path = "/kaggle/input/bird-call-spectrogram/spec_train/asbfly/XC175797.npy"
# image = np.load(npy_path)
# #Shorter image

# # Visualize the spectrogram
# plt.figure(figsize=(12, 4))
# plt.imshow(image[:48, :48], aspect='auto', origin='lower', cmap='viridis')
# plt.colorbar(label='Amplitude')
# plt.title("Bird Call Spectrogram ('48x48')")
# plt.xlabel('Time')
# plt.ylabel('Frequency')
# plt.show()


In [18]:
# height, width = 48, 48
# patch_size = 16

# image_batch = tf.expand_dims(tf.constant(image[:height, :width], dtype=tf.float32), 0)
# image_batch = tf.expand_dims(image_batch, axis=-1)
# print(image_batch.shape)
# patch_layer = Patches(patch_size)
# patches = patch_layer(image_batch)
# print(f"Image shape: {image_batch.shape}")  # Should be (1, 32, 32, 1)
# print(f"Patches shape: {patches.shape}")  # Should be (1, 4, 256)


# fig, axes = plt.subplots(1, 9, figsize=(16, 4))
# for patch_idx in range(9):  # You have 5 patches
#     # Get and reshape patch
#     flat_patch = patches[0, patch_idx, :]
#     patch_2d = tf.reshape(flat_patch, [patch_size, patch_size])
    
#     # Plot
#     axes[patch_idx].imshow(patch_2d, cmap='viridis')
#     axes[patch_idx].set_title(f'Patch {patch_idx}')
#     axes[patch_idx].axis('off')

# plt.tight_layout()
# plt.show()

In [19]:
# # Create reconstruction layer
# recon_layer = PatchReconstruction(patch_size=patch_size, 
#                                   img_height=height, 
#                                   img_width=width, 
#                                   channels=1)

# # Reconstruct the image
# reconstructed = recon_layer(patches)

# # Visualize the spectrogram
# plt.figure(figsize=(12, 4))
# plt.imshow(reconstructed[0, :, :, 0], aspect='auto', origin='lower', cmap='viridis')
# plt.colorbar(label='Amplitude')
# plt.title("Bird Call Spectrogram ('32x32')")
# plt.xlabel('Time')
# plt.ylabel('Frequency')
# plt.show()

# ViT Model

## Encoder

In [20]:
class TransformerEncoder(layers.Layer):
    def __init__(self, num_layers, hidden_dim, num_heads, dropout=0.1):
        super().__init__()
        self.blocks = []

        for _ in range(num_layers):
            self.blocks.append({
                "norm1": layers.LayerNormalization(epsilon=1e-6),
                "attn": layers.MultiHeadAttention(
                    num_heads=num_heads,
                    key_dim=hidden_dim // num_heads,
                    dropout=dropout,
                ),
                "add1": layers.Add(),
                "norm2": layers.LayerNormalization(epsilon=1e-6),
                "mlp1": layers.Dense(hidden_dim * 4, activation="gelu"),
                "mlp2": layers.Dense(hidden_dim),
                "add2": layers.Add(),
                "drop": layers.Dropout(dropout),
            })

    def call(self, x, training=False):
        for block in self.blocks:
            attn = block["attn"](
                block["norm1"](x),
                block["norm1"](x),
                training=training,
            )
            x = block["add1"]([x, attn])

            mlp = block["mlp2"](block["mlp1"](block["norm2"](x)))
            mlp = block["drop"](mlp, training=training)
            x = block["add2"]([x, mlp])

        return x


### Model

In [21]:
class VisionTransformer(keras.Model):
    def __init__(self, num_classes):
        super().__init__()

        self.patchify = Patches(CFG.patch_size)

        num_patches = (CFG.img_size[0] // CFG.patch_size) * \
                      (CFG.img_size[1] // CFG.patch_size)

        self.patch_encoder = PatchEncoder(
            num_patches=num_patches,
            projection_dim=192,
        )

        self.encoder = TransformerEncoder(
            num_layers=4,
            hidden_dim=192,
            num_heads=3,
        )

        self.norm = layers.LayerNormalization(epsilon=1e-6)
        self.pool = layers.GlobalAveragePooling1D()
        self.head = layers.Dense(num_classes, activation="softmax")

    def call(self, x, training=False):
        x = self.patchify(x)
        x = self.patch_encoder(x)
        x = self.encoder(x, training=training)
        x = self.norm(x)
        x = self.pool(x)
        return self.head(x)


# Training Loop

In [22]:
model = VisionTransformer(num_classes=num_classes)

model.compile(
    optimizer=keras.optimizers.Adam(CFG.learning_rate),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()


Model: "vision_transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ patches (Patches)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_encoder (PatchEncoder)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ ?                      │   0 (unbuilt) │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_8           │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CFG.epochs,
)

Epoch 1/30


I0000 00:00:1765907358.352734      63 service.cc:148] XLA service 0x798f540d0030 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765907358.353784      63 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765907358.353817      63 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765907359.998556      63 cuda_dnn.cc:529] Loaded cuDNN version 90300


   5/2446 ━━━━━━━━━━━━━━━━━━━━ 1:15 31ms/step - accuracy: 0.0000e+00 - loss: 5.4154

I0000 00:00:1765907368.626975      63 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2446/2446 ━━━━━━━━━━━━━━━━━━━━ 111s 34ms/step - accuracy: 0.0260 - loss: 4.7344 - val_accuracy: 0.0354 - val_loss: 4.5747
Epoch 2/30
2446/2446 ━━━━━━━━━━━━━━━━━━━━ 55s 22ms/step - accuracy: 0.0332 - loss: 4.5837 - val_accuracy: 0.0368 - val_loss: 4.5350
Epoch 3/30
2446/2446 ━━━━━━━━━━━━━━━━━━━━ 56s 22ms/step - accuracy: 0.0386 - loss: 4.5383 - val_accuracy: 0.0423 - val_loss: 4.5122
Epoch 4/30
2446/2446 ━━━━━━━━━━━━━━━━━━━━ 56s 22ms/step - accuracy: 0.0442 - loss: 4.4971 - val_accuracy: 0.0437 - val_loss: 4.4766
Epoch 5/30
2446/2446 ━━━━━━━━━━━━━━━━━━━━ 56s 22ms/step - accuracy: 0.0480 - loss: 4.4498 - val_accuracy: 0.0491 - val_loss: 4.4011
Epoch 6/30
2446/2446 ━━━━━━━━━━━━━━━━━━━━ 56s 22ms/step - accuracy: 0.0595 - loss: 4.3642 - val_accuracy: 0.0642 - val_loss: 4.3467
Epoch 7/30
2446/2446 ━━━━━━━━━━━━━━━━━━━━ 56s 22ms/step - accuracy: 0.0706 - loss: 4.2778 - val_accuracy: 0.0740 - val_loss: 4.2827
Epoch 8/30
2446/2446 ━━━━━━━━━━━━━━━━━━━━ 56s 22ms/step - accuracy: 0.0930 - loss: 4.1

# Experiment Configuration

In [24]:
# from dataclasses import dataclass

# @dataclass
# class ExperimentConfig:
#     """Configuration for MAE experiments"""
#     name: str
#     hidden_dim: int = 256
#     num_layers: int = 2
#     num_heads: int = 4
#     batch_size: int = 16
#     mask_ratio: float = 0.75
#     learning_rate: float = 0.001
#     epochs: int = 100
#     seed: int = 42
#     img_size: list = None
#     patch_size: int = 16
#     num_patches: int = None  # Will be auto-calculated
    
#     def __post_init__(self):
#         """Auto-calculate num_patches from img_size and patch_size"""
#         # Use defaults if not provided
#         if self.img_size is None:
#             self.img_size = [256, 944]
        
#         # Calculate num_patches
#         h_patches = self.img_size[0] // self.patch_size
#         w_patches = self.img_size[1] // self.patch_size
#         self.num_patches = h_patches * w_patches
        
#         print(f"Config '{self.name}':")
#         print(f"  Image size: {self.img_size}")
#         print(f"  Patch size: {self.patch_size}")
#         print(f"  Patches: {h_patches} x {w_patches} = {self.num_patches}")

In [25]:
# EXPERIMENTS = [
#     # Phase 1: MLP Baseline (no attention heads)
#     ExperimentConfig("mae_mlp_mask50", hidden_dim=256, num_layers=2, num_heads=0, 
#                      batch_size=64, mask_ratio=0.50),
#     ExperimentConfig("mae_mlp_mask75", hidden_dim=256, num_layers=2, num_heads=0, 
#                      batch_size=64, mask_ratio=0.75),
    
#     # Phase 2: Small Transformer
#     ExperimentConfig("mae_small_mask50_4_heads", hidden_dim=256, num_layers=2, num_heads=4, 
#                      batch_size=64, mask_ratio=0.50),
#     ExperimentConfig("mae_small_mask75_4_heads", hidden_dim=256, num_layers=2, num_heads=4, 
#                      batch_size=64, mask_ratio=0.75),
    
#     # Phase 3: Medium Transformer
#     ExperimentConfig("mae_medium_mask50_6_heads", hidden_dim=384, num_layers=4, num_heads=6, 
#                      batch_size=32, mask_ratio=0.50),
#     ExperimentConfig("mae_medium_mask75_6_heads", hidden_dim=384, num_layers=4, num_heads=6, 
#                      batch_size=32, mask_ratio=0.75),
    
#     # Phase 4: Large Transformer
#     ExperimentConfig("mae_large_mask50_8_heads", hidden_dim=512, num_layers=6, num_heads=8, 
#                      batch_size=16, mask_ratio=0.50),
#     ExperimentConfig("mae_large_mask75_8_heads", hidden_dim=512, num_layers=6, num_heads=8, 
#                      batch_size=16, mask_ratio=0.75),
# ]

In [26]:
# K.clear_session()
# # model, history = train_true_mae_experiment(EXPERIMENTS[0], train_ds, val_ds)

In [27]:
# model, history = train_true_mae_experiment(EXPERIMENTS[1], train_ds, val_ds)

In [28]:
# model, history = train_true_mae_experiment(EXPERIMENTS[2], train_ds, val_ds)

In [29]:
# model, history = train_true_mae_experiment(EXPERIMENTS[3], train_ds, val_ds)

In [30]:
# model, history = train_true_mae_experiment(EXPERIMENTS[4], train_ds, val_ds)

In [31]:
# model, history = train_true_mae_experiment(EXPERIMENTS[5], train_ds, val_ds)